In [1]:
import math 
import copy
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import mpl

##可见这里的每个高斯模型都是二维的，且相互独立
##mu.shape = (k, 2)
##sigma.shape = (k, 2, 2)
##alpha.shape = (4,)

def generate_paramters(k):
    """
    EM算法的各个子高斯混合模型的初始参数
    ：param k:混合高斯模型个数
    ------
    ：return: 各个子高斯模型的初始参数
    
    """
    mu_ = []
    sigma_ =[]
    alpha_ = np.random.random(k)
    alpha_ = alpha_ / np.sum(alpha_)
    for i in range(k):
        mu_.append(np.random.randint(5, 45, size = (2, )).tolist())
        sigma_.append(np.asmatrix(np.diag(np.random.randint(10, 50, size = (2, )))))
        
    mu_ = np.asmatrix(mu_)
    return (mu_, sigma_, alpha_)


def generate_spec_paramters():
    mu_ = np.matrix([
        [5, 35],
        [30, 40],
        [20, 20],
        [45, 15]
    ])
    sigma_ = [
        [np.matrix([[30, 0], [0, 30]])],
        [np.matrix([[30, 0], [0, 30]])],
        [np.matrix([[30, 0], [0, 30]])],
        [np.matrix([[30, 0], [0, 30]])]
    ]
    alpha_ = [0.25, 0.2, 0.25, 0.3]
    return (mu_, sigma_, alpha_)

mu_, sigma_, alpha_ = generate_paramters(4)

print(mu_)
print(sigma_)
print(alpha_)


[[33 44]
 [26 39]
 [39 14]
 [39 26]]
[matrix([[17,  0],
        [ 0, 14]]), matrix([[21,  0],
        [ 0, 21]]), matrix([[28,  0],
        [ 0, 14]]), matrix([[26,  0],
        [ 0, 36]])]
[0.03590464 0.2602556  0.38489952 0.31894024]


In [2]:
def generate_data(N, k, mu_, sigma_, alpha_):
    """
    高斯混合模型生成观测样本，以及EM算法求解高斯混合模型的初始参数
    :param N: 观测样本数量
    :param k: 混合的子高斯模型的个数
    :param mu_: 生成观测样本的各个子高斯模型的均值
    :param sigma_: 生成观测样本的各个子高斯模型的协方差
    :param alpha_: 生成观测样本的各个子高斯模型的权值系数
    """
    ##EM算法求解高斯混合模型的初始化参数
    global expect ##期望第i个样本属于第j个模型的概率的期望
    expect = np.zeros((N, k))
    
    ##生成可观测数据
    global X
    X = np.zeros((N, 2))
    X = np.matrix(X)
    for i in range(N):
        rand_num = np.random.random(1)
        if rand_num < alpha_[0]:
            ##使用第一个高斯模型生成2维数据
            X[i, :] = np.random.multivariate_normal(mu_[0, :].tolist()[0], sigma_[0], 1)
            
        elif alpha_[0] <= rand_num < alpha_[0] + alpha_[1] :
            ##使用第二个高斯模型生成2维数据
            X[i, :] = np.random.multivariate_normal(mu_[1, :].tolist()[0], sigma_[1], 1)
            
        elif alpha_[0] + alpha_[1] <= rand_num < alpha_[0] + alpha_[1] + alpha_[3] :
            ##使用第三个高斯模型生成2维数据
            X[i, :] = np.random.multivariate_normal(mu_[2, :].tolist()[0], sigma_[2], 1)
            
        else:
            ##使用第四个高斯模型生成2维数据
            X[i, :] = np.random.multivariate_normal(mu_[3, :].tolist()[0], sigma_[3], 1)
        
    print("可观测的数据： \n", X)
    print("EM算法的初始化 mu：\n", mu_)
    print("EM算法的初始化 sigma：\n", sigma_)
    print("EM算法的初始化 alpha：\n", alpha_)
    
    
generate_data(100, 4, mu_, sigma_, alpha_)    



可观测的数据： 
 [[38.6682229  42.97791978]
 [39.11982864 27.26948759]
 [36.05943095 15.74567775]
 [33.38395444 22.91237779]
 [37.13304922  8.15293418]
 [41.92624211 15.08326188]
 [32.03184123 40.92457973]
 [26.57160203 45.7096214 ]
 [36.87717596 15.7251456 ]
 [36.12856697 18.85593084]
 [45.51359119 14.16726195]
 [33.72451577  9.42783898]
 [30.22669848 13.1976292 ]
 [45.25913544 28.44327876]
 [48.92222643 27.50895756]
 [30.92671157 39.40589912]
 [34.11276679 19.94822623]
 [39.10290251 33.7829003 ]
 [21.68293415 31.60064822]
 [35.49158224 47.50332914]
 [19.84071034 39.66671443]
 [36.6559042  14.75275819]
 [41.60835544 10.0402429 ]
 [40.96428153  8.43609007]
 [35.64197941 33.72378704]
 [44.15528738 14.01437979]
 [39.83803405 12.60757626]
 [40.76908454 25.28959016]
 [18.09883735 45.21718193]
 [48.62891021 31.45066718]
 [29.52624815 41.23244767]
 [33.18331796 24.67357107]
 [26.92690478 32.28105983]
 [37.86934594 32.49787597]
 [28.57420316 43.79231866]
 [41.98604489 11.09408269]
 [38.94304595 22.6

In [4]:
def e_step(k, N):
    """
    EM算法的E步:求期望
    :param k: 混合的高斯模型个数
    :param N: 观测样本个数
    
    """
    global X  ##X.shape = (N, j)
    global mu
    global sigma
    global expect
    global alpha
    
    for i in range(N):
        denom = 0
        for j in range(k):
            ##求第j个高斯模型
            denom += alpha[j] * math.exp(-(X[i, :] - mu[j, :]) * sigma[j].I * np.transpose(X[i, :] - mu[j, :])) /np.sqrt(np.linalg.det(sigma[j]))
                                         ##乘以协方差矩阵的逆
                                         #* sigma[j].I 
                                         ##x-u的转置
                                         #* np.transpose(X[i, :] - mu[j, :]))
                                         ##除以sigma的行列式的根号
                                         #/np.sqrt(np.linalg.det(sigma[j]))
        for j in range(k):
            numer = math.exp(-(X[I, :] - mu[j, :]) * sigma[j].I * np.transpose(X[I, :] - mu[j, :])) / np.sqrt(np.linalg.det(sigma[j]))
                             #* sigma[j].I * 
                             #np.transpose(X[I, :] - mu[j, :]))
                            #/ np.sqrt(np.linalg.det(sigma[j]))
            ##求期望
            expect[i, j] = alpha[j] * numer / denom
    print("隐变量概率: \n", expect)
        
def m_step(k, N):
    """
    EM算法的M步：求最大值
    :param k: 混合的高斯模型个数
    :param N: 观测样本个数
    """
    global X
    global mu
    global sigma 
    global expect
    global alpha
    
    for j in range(k):
        denom = 0
        numer = 0
        for i in range(N):
            numer += expect[i, j]*X[i, :]
            denom += expect[i, j]
        
        ##更新混合项系数
        alpha[j] = denom / N
        ##更新求均值
        mu[j, :] = numer / denom
        ##更新协方差矩阵
        numer = 0
        for i in range(N):
            numer += np.transpose(X[i, :] - mu[j, :]) * (X[i, :] - mu[j, :]) * expect[i, j]
        sigma[j] = np.asmatrix(numer / denom)
    
    
    
    

IndentationError: unexpected indent (<ipython-input-4-82d7c275fc4d>, line 24)

In [ ]:
if __name__ = "__main__":
    """
    1.初始化参数时，必须保证sigma的行列式大于0，否则高斯函数计算中对sigma行列式开方会报错"OverflowError"
    2.如果报错，重新运行一遍
    """
    threshold = 0.0001
    iter_num = 60
    N = 500
    k = 4
    probility np.zeros(N)
    ##方法一：获取手动指定的混合高斯模型参数
    (mu, sigma, alpha) = generate_spec_paramters()
    ##方法二：获取随机生成的混合高斯模型参数
    #(mu_, sigma_, alpha_) = generate_paramters(k)
    
    ##获取从混合高斯模型中采样得到的数据: X
    generate_data(N, k, mu, sigma, alpha)
    
    for i in range(iter_num):
        ##均值误差
        err_mu = 0 
        ##协方差误差
        err_sigma = 0
        ##混合项系数误差
        err_alpha = 0
        old_mu = copy.deepcopy(mu)
        old_sigma = copy.deepcopy(sigma)
        old_alpha = copy.deepcopy(alpha)
        e_step(k, N)
        m_step(k, N)
        print("\n------------------------------------")
        print("迭代次数：", i + 1)
        print("估计的mu：\n", mu)
        print("估计的sigma：\n", sigma)
        print("估计的混合项系数alpha: \n", alpha)
        for z in range(k):
            err_mu += np.sum(abs(old_mu[z] - mu[z]))
            err_sigma += nu.sum(np.sum(abs(old_sigma[z] - sigma[z]), 0), 1)
            err_alpha += abs(old_alpha[z] - alpha[z])
        ##达到迭代精度就退出迭代
        if (err_mu <= threshold) and (err_alpha < threshold) and (err_sigma <= threshold):
            print("参数误差(err_mu, err_sigma, err_alpha) : (%lf, %lf, %lf) "%(err_mu, err_sigma, err_alpha))
            break
            
    font_path = "C:/Windows/Font/ssimkai.ttf"
    myfont = mpl.font_manager.FontProperties(fname = font_path)
    ##-------------------1.可视化原始数据
    fig = fig.figure()
    ax = fig.add_subplot(111)
    ax.scatter(X[:, 0].tolist(), X[:, 1].tolist(), c = 'b', s = 25, alpha = 0.4, marker = 'o')
    ax.set_title("四个高斯模型生成的原始数据", fontproperties = myfont)
    fig.savefig("./figure/1_original_data.png", dpi = 800)
    fig.show()
    
    ##-------------------2.可视化EM算法训练的高斯模型对数据的分类结果
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title("EM算法分类数据", fontproperties = myfont)
    order = np.zeros(N)
    color = ['b', 'r', 'k', 'y']
    for i in range(N):
        ##选出样本分别属于哪个高斯模型
        order[i] = np.argmax(expect[i, :])
        ##计算混合高斯概率
        for j in range(k):
            probility[i] += alpha[int(orfer[i])] * math.exp(-(X[i, :] - mu[j, :]) 
                                                            * sigma[j].I 
                                                            * np.transpose(X[i, :] - mu[j, :]))
                                                            /(np.sqrt(np.linalg.det(sigma[j])) * 2 * np.pi)
            
        ##绘制分类后的散点图
        ax.scatter(X[i, 0], X[i, 1], c = color[int(order[i])], s = 25, alpha = 0.4, marker = 'o')
    fig.savefig("./figure/2_classified_data.png", dpi = 800)
    fig.show()
    
    ##-------------------3.绘制观测样本概率密度三维图像
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    ax.set_title("观测样本概率密度3D视图", fontproperties = myfont)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("p")
    for i in range(N):
        ax.scatter(X[i, 0], X[i, 1], probility[i], c = color[int(order[i])])
    fig.savefig("./figure/3_probability_3D.png", dpi = 800)
    fig.show()
    
    
    
        